In [1]:
import os
import sys
import tensorflow as tf

from sklearn.model_selection import train_test_split

sys.path.insert(0, r"../utilities")

from utils import *
from params import *
from Seq2SeqAttentionTrainer import Seq2SeqAttentionTrainer

tf.__version__

'2.0.0'

In [2]:
    data_dir = "../data"
    # reading data

    en_lines, fr_lines = read_data_files(data_dir, ("small_vocab_en", "small_vocab_fr"))
    """
    data = read_data(os.path.join(data_dir, "fra-eng"), "fra.txt")
    en_lines, fr_lines = list(zip(*data))
    
    en_lines = en_lines[:30000]
    fr_lines = fr_lines[:30000]
    """
    en_lines = [normalize(line) for line in en_lines]
    fr_lines = [normalize(line) for line in fr_lines]

    en_train, en_test, fr_train, fr_test = train_test_split(en_lines, fr_lines, shuffle=True, test_size=0.1)

    fr_train_in = ['<start> ' + line for line in fr_train]
    fr_train_out = [line + ' <end>' for line in fr_train]

    fr_test_in = ['<start> ' + line for line in fr_test]
    fr_test_out = [line + ' <end>' for line in fr_test]

    fr_tokenizer = Tokenizer(filters='')
    en_tokenizer = Tokenizer(filters='')

    input_data = [fr_train_in, fr_train_out, fr_test_in, fr_test_out, fr_test, fr_train]
    fr_train_in, fr_train_out, fr_test_in, fr_test_out, fr_test, fr_train = tokenizeInput(input_data,
                                                                                          fr_tokenizer)

    input_data = [en_train, en_test]
    en_train, en_test = tokenizeInput(input_data, en_tokenizer)

    en_vocab_size = len(en_tokenizer.word_index)+1
    fr_vocab_size = len(fr_tokenizer.word_index)+1
    print("en_vocab {}\nfr_vocab {}" .format(en_vocab_size, fr_vocab_size))

reading data from  ../data/small_vocab_en
reading data from  ../data/small_vocab_fr
en_vocab 203
fr_vocab 336


In [3]:
trainer = Seq2SeqAttentionTrainer(BATCH_SIZE, LSTM_SIZE, EMBEDDING_SIZE, 1)

In [4]:
losses, accuracy= trainer.train([en_train, fr_train_in, fr_train_out], [en_test, fr_test_in, fr_test_out], [en_tokenizer, fr_tokenizer], 20, "concat")

en_vocab 203
fr_vocab 336
Number of devices: 4
creating dataset...
dataset created
decoder_output  Tensor("decoder/Attenton_W_s/BiasAdd:0", shape=(32, 336), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:0)
fr_data_out[:,i]  Tensor("strided_slice_1:0", shape=(32,), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:0)
INFO:tensorflow:Error reported to Coordinator: in converted code:

    /shared/NLP/Seq2SeqAttention/Seq2SeqAttentionTrainer.py:110 train_step  *
        loss +=compute_loss(decoder_output, fr_data_out[:,i])
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_ops.py:924 r_binary_op_wrapper
        x = ops.convert_to_tensor(x, dtype=y.dtype.base_dtype, name="x")
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:1184 convert_to_tensor
        return convert_to_tensor_v2(value, dtype, preferred_dtype, name)
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:12

TypeError: in converted code:

    /shared/NLP/Seq2SeqAttention/Seq2SeqAttentionTrainer.py:122 distributed_train_step  *
        per_replica_losses = self.strategy.experimental_run_v2(train_step,
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/distribute/distribute_lib.py:760 experimental_run_v2
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /shared/NLP/Seq2SeqAttention/Seq2SeqAttentionTrainer.py:110 train_step  *
        loss +=compute_loss(decoder_output, fr_data_out[:,i])
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_ops.py:924 r_binary_op_wrapper
        x = ops.convert_to_tensor(x, dtype=y.dtype.base_dtype, name="x")
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:1184 convert_to_tensor
        return convert_to_tensor_v2(value, dtype, preferred_dtype, name)
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:1242 convert_to_tensor

In [ ]:
#training with distribution
#distributedTrain(en_data, fr_data, en_test, fr_test, fr_tokenizer, en_tokenizer, attention_type="concat")